In [6]:
import random
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tkinter import Tk
import os
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

In [7]:
service = Service(r'C:\Users\Nick\Documents\GitHub\grvmodel\DATA\GRV_scraper\gdriver\geckodriver.exe')


In [8]:

driver = Firefox(service=service)  
url = 'https://www.tab.com.au'

meets_list = []
driver.get(url)

In [9]:
menu = driver.find_element(By.CLASS_NAME, 'toggle-menu-link')

In [10]:
menu.click()

In [11]:
menu2 = driver.find_element(By.XPATH, '/html/body/ui-view/header/div/div[1]/div[2]/div[2]/nav[1]/div[1]/div[1]')
menu2.click()

In [12]:
greys = driver.find_element(By.XPATH, "/html/body/ui-view/header/div/div[1]/div[2]/div[2]/simple-main-nav-menu[1]/nav/div/div[3]/a")

In [13]:
greys.click()

In [14]:
driver.refresh()

In [15]:
race_elements = driver.find_elements(By.CLASS_NAME, '_1pb745v')

In [16]:
race_elements = driver.find_elements(By.XPATH,"//*[@data-testid='race']")

In [17]:
tab_dict = {}
tab_dict['links'] = []
tab_dict['children'] = []
tab_dict['children2'] = []
tab_dict['children_id'] = []
tab_dict['children_text'] = []
# tab_dict['child1'] = []
# tab_dict['child2'] = []
for r in race_elements:
    # print(r)
    tab_dict['links'].append(r.get_attribute("href"))
    children = r.find_elements_by_xpath('./child::*')
    children2 = [c.find_elements_by_xpath('./child::*') for c in children]
    tab_dict['children2'].append([c.get_attribute('class') for sublist in children2 for c in sublist])
    tab_dict['children_id'].append([c.get_attribute('data-testid') for sublist in children2 for c in sublist])
    tab_dict['children_text'].append([c.text for sublist in children2 for c in sublist])
    tab_dict['children'].append([c.get_attribute('class') for c in children])

C:\Users\Nick\AppData\Local\Temp\ipykernel_9904\1746305968.py:12: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  children = r.find_elements_by_xpath('./child::*')
C:\Users\Nick\AppData\Local\Temp\ipykernel_9904\1746305968.py:13: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  children2 = [c.find_elements_by_xpath('./child::*') for c in children]


In [18]:
tab_df = pd.DataFrame(tab_dict)

In [19]:
tab_df['splits'] = tab_df.links.apply(lambda x: x.split('/'))
tab_df['date'] = tab_df.splits.apply(lambda x: x[4])
tab_df['track'] = tab_df.splits.apply(lambda x: x[5])
tab_df['race_num'] = tab_df.splits.apply(lambda x: x[8])

In [24]:
tab_df['odds_available'] = tab_df.children_id.apply(lambda x: True if 'icon-fixed-odds' in x else False)

In [25]:
tab_df['odds_available']

0      False
1      False
2      False
3      False
4      False
       ...  
198    False
199    False
200    False
201    False
202    False
Name: odds_available, Length: 203, dtype: bool

In [27]:
tab_df_avail = tab_df[tab_df['odds_available']]

In [34]:
race_link = tab_df_avail.links.iloc[-1]

In [33]:
driver.get(race_link)

In [23]:
tab_df.children_text[0]

['R1', '18:57', '3 4 7 8']

In [125]:
tab_df

,links,children
0,https://www.tab.com.au/racing/2023-05-13/NEWCA...,"[, _i9ed2x ]"
1,https://www.tab.com.au/racing/2023-05-13/NEWCA...,"[, _i9ed2x ]"
2,https://www.tab.com.au/racing/2023-05-13/NEWCA...,"[, _i9ed2x ]"
3,https://www.tab.com.au/racing/2023-05-13/NEWCA...,"[, _i9ed2x , _1j1uuf5 ]"
4,https://www.tab.com.au/racing/2023-05-13/NEWCA...,"[, _i9ed2x , _1j1uuf5 ]"
...,...,...
131,https://www.tab.com.au/racing/2023-05-13/SWIND...,"[, _i9ed2x ]"
132,https://www.tab.com.au/racing/2023-05-13/SWIND...,"[, _i9ed2x ]"
133,https://www.tab.com.au/racing/2023-05-13/SWIND...,"[, _i9ed2x ]"
134,https://www.tab.com.au/racing/2023-05-13/SWIND...,"[, _i9ed2x ]"
